In [1]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
from collections import defaultdict
import pandas as pd
import numpy as np
from konlpy.tag import Mecab
import re

# csv 파일 불러오기 & 전처리

In [2]:
old_poetry = pd.read_csv('Old_Poetry.csv')
Old_Poetry = old_poetry['본문'].values.tolist()
love_poem = pd.read_csv('Poem_Love.csv',lineterminator='\n')
Love_Poem = love_poem['본문'].values.tolist()
clean_lovepoem = []
for j in Love_Poem:
	#print(j)
	cleanpoem = re.sub('[…().!?,""---―\ufeff\u200bknsjQuery1319700039250="65">jQuery1319700039250="68">\r\xa0~‘’;/]','',str(j))
	edited_poem = re.sub('[\u2e80-\u2eff\u31c0-\u31ef\u3200-\u32ff\u3400-\u4dbf\u4e00-\u9fbf\uf900-\ufaff]', '', cleanpoem)
	clean_lovepoem.append(edited_poem)
print(clean_lovepoem[:2])
clean_oldpoem = []
for i in Old_Poetry:
	cleanPoem = re.sub('[…().!?,---―]','',i)
	edited_poem = re.sub('[\u2e80-\u2eff\u31c0-\u31ef\u3200-\u32ff\u3400-\u4dbf\u4e00-\u9fbf\uf900-\ufaff]', '', cleanPoem)
	clean_oldpoem.append(edited_poem)

['가족의 힘   솔새김남식\n\n이 세상에서 가장 소중한 사람은 \n누구냐고 질문하면 \n물어보나 마나 누구든지 \n바로 가족이라고 답 할 것이다 \n곤히 잠든 식구를 바라보면서 \n가족의 소중함을 다시 한 번 깨닫고 \n밖에 나가 일하고 돌아와서\n곤히 잠든 남편 \n날 낳아서 길러 주신 부모님 \n뛰어 놀다 쓰러져 자는 사랑스러운 아이들 \n그리고 사랑을 듬뿍 안겨주는 \n내 아내 모두가 \n소중한 사람입니다 \n내 곁에는 이렇게 소중한 가족이 있구나 \n그럴 때면 한결 힘이 납니다 \n아무리 삶이 힘들고  \n버는 돈이 시원찮아도 가족이 있으면 \n힘이 납니다', '애틋한 마음  솔새김남식\n애틋한 마음이란 \n그 사람이 \n나를 만나고 돌아 갈 때\n그 사람이 \n보이지 \n않을 때 까지 \n그 자리에 서 있는 것\n그리고 \n돌아와서 \n문자를 보내는 것']


/home/ubuntu/anaconda3/envs/Minseok/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Possible set difference at position 10
  
/home/ubuntu/anaconda3/envs/Minseok/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Possible set difference at position 8
  


In [3]:
data_list = []
for i in clean_oldpoem:
	data = i.split('\n \n')
	data_list.append(data)
data = []
for j in data_list:
	for k in j:
		cleaned_datalist = re.sub('[\\n]','',k)
		data.append(cleaned_datalist)
data[:5]

['가을에는기도하게 하소서 낙엽들이 지는 때를 기다려 내게 주신겸허한 모국어로 나를 채우소서',
 '가을에는사랑하게 하소서 오직 한 사람을 택하게 하소서가장 아름다운 열매를 위하여 이 비옥한시간을 가꾸게 하소서',
 '가을에는호올로 있게 하소서 ',
 '나의 영혼굽이치는 바다와백합의 골짜기를 지나마른 나뭇가지 위에 다다른 까마귀같이 ',
 '지상에는아홉 켤레의 신발아니 현관에는 아니 들깐에는아니 어느 시인의 가정에는알전등이 켜질 무렵을문수가 다른 아홉 켤레의 신발을']

# 핵심키워드 추출(TF-IDF) 220명 시인

In [4]:
mecab = Mecab()
preprocessed_docs = []
for i in data:
  token_list = [] 
  for token in mecab.pos(i):
    if token[1] in ['NNG']:
      token_list.append(token[0])
  preprocessed_docs.append(" ".join(token_list))
display(preprocessed_docs[:10],len(preprocessed_docs))

['가을 기도 낙엽 때 겸허 모국어',
 '가을 사랑 사람 열매 시간',
 '가을',
 '영혼 바다 백합 골짜기 나뭇가지 위 까마귀',
 '지상 신발 현관 시인 가정 알전 문수 신발',
 '신발 구문 반눈 얼음 길 옆 코 귀염 귀염 막내',
 '미소 얼굴 얼음 눈 벽 지상 연민 삶 길 신발 구문 반',
 '아랫목 강아지 강아지 굴욕 굶주림 길 아버지 구문 반 신발 지상 아버지 존재 미소 얼굴',
 '문 안 생활 까닭 밤 꾸지람 집내 문패 앞 밤 속 제웅 만감 식구 호어 한구석 수입 지붕 서리 침 월광 집 힘 도장 수명 전당 문고리 쇠사슬 문 문',
 '바닷가 햇빛 바위 우 간']

860

In [5]:
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(preprocessed_docs)
word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx
keywords_lists = []
for i, sent in enumerate(preprocessed_docs):
    print('====== document[%d] ======' % i)
    for token in sent.split():
        matrix = sp_matrix[i, word2id[token]]
        print(token,matrix)
        if 0.3 <= matrix < 1.0:
            keywords_lists.append(token)
len(keywords_lists)
    #print( [ (token, sp_matrix[i, word2id[token]]) for token in sent.split() ] )

/home/ubuntu/anaconda3/envs/Minseok/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


====== document[0] ======
가을 0.31941177667068965
기도 0.4364342697148082
낙엽 0.45227309424538903
때 0.0
겸허 0.5014729770208347
모국어 0.5014729770208347
====== document[1] ======
가을 0.4200259286344563
사랑 0.3752629909867417
사람 0.37751223566537756
열매 0.5092125427206432
시간 0.5300405693240402
====== document[2] ======
가을 1.0
====== document[3] ======
영혼 0.3835660956031695
바다 0.26667980393203156
백합 0.4873112114088536
골짜기 0.39169028834770964
나뭇가지 0.4593438842712976
위 0.0
까마귀 0.424109219082059
====== document[4] ======
지상 0.2985062739791317
신발 0.5970125479582634
현관 0.3429905492287079
시인 0.2896548886772586
가정 0.3429905492287079
알전 0.3429905492287079
문수 0.3429905492287079
신발 0.5970125479582634
====== document[5] ======
신발 0.3056918959987403
구문 0.31678589259141976
반눈 0.35124699359150796
얼음 0.2823247915913315
길 0.0
옆 0.0
코 0.0
귀염 0.7024939871830159
귀염 0.7024939871830159
막내 0.33108854177289204
====== document[6] ======
미소 0.3900304949559761
얼굴 0.26843348200924194
얼음 0.360216543663832
눈 0.0
벽 0.0
지상 0.3900

3121

In [6]:
keywords_oldpoem = []
for v in keywords_lists:
    if v not in keywords_oldpoem:
        keywords_oldpoem.append(v)
# tests = []
# for i,j in enumerate(keywords_oldpoem):
#     if i < 5:
#         tests.append(j)
display(keywords_oldpoem[:5],len(keywords_oldpoem))

['가을', '기도', '낙엽', '겸허', '모국어']

1556

# TF-IDF 또 다른 방법

In [9]:
# count_vect = CountVectorizer(max_df=0.85, max_features=10000)
# word_count = count_vect.fit_transform(preprocessed_docs)
# print((count_vect.get_feature_names()[:10]))
# tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
# tfidf_transformer.fit(word_count)

In [10]:
# def sort_keywords(keywords):
#     return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)
# def extract_keywords(feature_names, sorted_keywords, n=5):
#     return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]
# tfidf_vect = tfidf_transformer.transform(count_vect.transform([preprocessed_docs[0]]))
# feature_names = count_vect.get_feature_names()
# sorted_keywords = sort_keywords(tfidf_vect.tocoo())
# keywords = extract_keywords(feature_names, sorted_keywords)
# print(keywords)
# print("\n===== 원문 =====")
# print(Old_Poetry[0][:100])
# print("\n=== 핵심키워드 ===")
# keywords_lists = []
# for k in keywords:
#     print(k)